# Gradient Boosted Tree Model with XGBoost

In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score, classification_report
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree
from xgboost import to_graphviz
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import numpy as np
import graphviz
import joblib

In [42]:
X = pd.read_csv("../../Data/preprocessed_training_dataset.csv", index_col="respondent_id")
y = pd.read_csv("../../Data/training_set_labels.csv", index_col="respondent_id", usecols=["respondent_id", "h1n1_vaccine", "seasonal_vaccine"])

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [44]:
# Espacio de búsqueda de hiperparámetros
search_space = {
    'estimator__max_depth': Integer(3, 10),
    'estimator__learning_rate': Real(0.01, 0.3, prior='log-uniform'),
    'estimator__subsample': Real(0.5, 1.0),
    'estimator__colsample_bytree': Real(0.5, 1.0),
    'estimator__n_estimators': Integer(50, 300),
    'estimator__gamma': Real(0.0, 1.0)
}

# Crear el modelo base
xgb_model = XGBClassifier(eval_metric='logloss', random_state=42)

# Envolverlo en MultiOutputClassifier para predicción multietiqueta
multi_xgb = MultiOutputClassifier(xgb_model)

In [ ]:
# Configurar BayesSearchCV
opt = BayesSearchCV(
    multi_xgb,
    search_space,
    cv=5,
    n_iter=50,
    scoring='roc_auc',
    random_state=42,
    n_jobs=-1
)

# Entrenar el modelo con optimización
opt.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros:", opt.best_params_)

# Guardar el mejor modelo optimizado
joblib.dump(opt.best_estimator_, "XGBoost_best_model.pkl")
print("Modelo guardado como 'XGBoost_best_model.pkl'")

Mejores hiperparámetros: OrderedDict({'estimator__colsample_bytree': 0.5, 'estimator__gamma': 0.0, 'estimator__learning_rate': 0.04351555474891697, 'estimator__max_depth': 5, 'estimator__n_estimators': 300, 'estimator__subsample': 0.5})


In [46]:
# Predicción
y_pred = opt.best_estimator_.predict(X_test)

# Calcular probabilidades para AUROC
y_proba = opt.best_estimator_.predict_proba(X_test)

# Convertir las probabilidades a un DataFrame para cada etiqueta
y_proba_h1n1 = y_proba[0][:, 1]  # Para H1N1
y_proba_seasonal = y_proba[1][:, 1]  # Para vacuna estacional

# Extraer las etiquetas reales como arrays 1D
y_test_h1n1 = y_test.iloc[:, 0].values  # H1N1
y_test_seasonal = y_test.iloc[:, 1].values  # Vacuna estacional

# AUROC para cada etiqueta
roc_auc_h1n1 = roc_auc_score(y_test_h1n1, y_proba_h1n1)
roc_auc_seasonal = roc_auc_score(y_test_seasonal, y_proba_seasonal)

print(f"AUROC para H1N1: {roc_auc_h1n1}")
print(f"AUROC para vacuna estacional: {roc_auc_seasonal}")

# Informe de clasificación
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

AUROC para H1N1: 0.8452206255778862
AUROC para vacuna estacional: 0.862288951930725
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.68      0.46      0.55      1135
           1       0.78      0.76      0.77      2488

   micro avg       0.76      0.66      0.71      3623
   macro avg       0.73      0.61      0.66      3623
weighted avg       0.75      0.66      0.70      3623
 samples avg       0.35      0.33      0.33      3623



c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VENV\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VENV\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VEN

In [ ]:
# Cargar el dataset de prueba preprocesado
test_set_preprocessed = pd.read_csv("../../Data/preprocessed_test_dataset.csv")

# Asegurarse de que respondent_id esté disponible
respondent_ids = test_set_preprocessed['respondent_id']

# Eliminar respondent_id del conjunto de características
X_test_final = test_set_preprocessed.drop(columns=['respondent_id'])

# Realizar predicciones de probabilidad
y_proba_test = opt.best_estimator_.predict_proba(X_test_final)

# Extraer las probabilidades para la clase positiva (1)
y_proba_h1n1_test = y_proba_test[0][:, 1]  # Probabilidades para H1N1
y_proba_seasonal_test = y_proba_test[1][:, 1]  # Probabilidades para vacuna estacional

# Crear el DataFrame de submission
submission = pd.DataFrame({
    "respondent_id": respondent_ids,
    "h1n1_vaccine": y_proba_h1n1_test,
    "seasonal_vaccine": y_proba_seasonal_test
})

# Guardar el archivo de submission
submission.to_csv("XGBoost_submission.csv", index=False)
print("Archivo 'XGBoost_submission.csv' generado correctamente.")

Archivo 'submission.csv' generado correctamente.


Ejemplo de carga de modelo guardado para su posterior uso.

In [ ]:
# Cargar el modelo guardado
loaded_model = joblib.load("best_model.pkl")
print("Modelo cargado correctamente.")

# Realizar predicciones usando el modelo cargado
y_proba_test = loaded_model.predict_proba(X_test_final)

# Extraer probabilidades para cada etiqueta
y_proba_h1n1_test = y_proba_test[0][:, 1]  # Probabilidades para H1N1
y_proba_seasonal_test = y_proba_test[1][:, 1]  # Probabilidades para vacuna estacional

# Crear un DataFrame para la submission si es necesario
submission = pd.DataFrame({
    "respondent_id": respondent_ids,
    "h1n1_vaccine": y_proba_h1n1_test,
    "seasonal_vaccine": y_proba_seasonal_test
})